## Motivation

Once single-cell genomics data has been processed, one can dissect important relationships between observed features in their genome context. In our genome, the activation of genes is controlled in the nucleus by the RNA transcriptional machinery, which activates local (promoters) or distal cis-regulatory elements (enhancers), to control the amount of RNA produced by every gene.

Conceptually, a Gene Regulatory Network (GRN) refers to a best-educated graph representation of how certain genes that control transcription i.e. "**Transcription Factors**" (TF) are in charge of directly controlling the transcription rates of their target genes (cis-regulation). At the same time, such target genes once activated, can be in charge of controlling other downstream target genes (trans-regulation). Computationally, methods that infer GRNs consider the co-variation of RNA and ATAC features to identify modules that could be grouped and associated simultaneously with a few TFs. A group of genes controlled by the activity of the same TF is defined as a **regulon**.

In addition to co-variation, several methods have recognized the gathering and injection of prior knowledge data, such as the locations where a TF bind in the genome, or previously reported TF-target gene associations, to pre-define gene-gene edges that guide the inference of GRNs that are most-supported by that type of evidence. To date, benchmarking of GRNs differs from other Machine Learning tasks such as image inference, in such a way that labeled data is sparse and hard to validate. For this reason, several methods have developed their own benchmark that compares new methods against others, using majorly real data. The generalization of GRN inference methods is an active discussion topic in regulatory genomics, and differently from previous chapters, requires a gold standard that is at the moment difficult to agree on and be reused consistently by the community.

The purpose of this chapter is to showcase the general pipeline of how a GRN can be generated, using methods that have been to our knowledge presented in a way that allows a validation with a low amount of software dependencies. Due to the benchmarking limitations of GRN tools in general, we recommend these tools but we cannot say that they will perform best in all possible scenarios. We would rather recommend those as a starting point, given available data, to then explore their generated GRN representations with the lowest computational effort.

#### Gathering TF regulons from public data

TF- Regulons have been annotated in academic studies, databases that compiled those, and also consortia efforts such as ENCODE. As general recommendations, one can inspect [TTRUST](https://www.grnpedia.org/trrust/) {cite}`at:Han2015-yb`, [DoRothEA](https://saezlab.github.io/dorothea/) {cite}`at:Garcia-Alonso2019-ly`, [KnockTF](http://www.licpathway.net/KnockTFv2/index.php) {cite}`at:Feng2020-nl`, among others, for eukaryotic TF-regulons. For prokaryotes [RegulonDB](https://regulondb.ccg.unam.mx/) {cite}`at:Santos-Zavaleta2018-be` is a known and recognized database.

#### **Limitations of TF regulons**

The source and confidence of a TF-regulon have some limitations, such as data source and experimental readout. If the data source for a regulon is not matched to the cell type of interest, then results can not be put into the context of the particular biological system. Alternatively, if the experimental readout is not measuring *cis* regulation events, directly provoked by the TF of interest, our TF-regulon might contain *trans* regulation events. The usage of a TF regulon gathered in an equivalent biological system is strongly recommended. However, if those are not available the interpretation of results might be biased due to the forcing of priors.

## **This chapter: Generation of GRNs using RNA data**

We will explore scRNA-seq data and predicted TF-regulons, using the tool SCENIC. Specifically, we will execute SCENIC {cite}`Aibar2017-tp` on a donor of the NeurIPS 2021 dataset, and we will interpret the results. The main processing steps described in this notebook are adapted based on SCENIC's core tutorials [Tutorial](https://github.com/aertslab/SCENICprotocol/blob/master/notebooks/PBMC10k_SCENIC-protocol-CLI.ipynb)

### **Outcome of the analysis**

This notebook allows interpreting single-cell RNA-seq data through the inference of a gene regulatory network, to get potential associations between Transcription Factors and target genes that explain gene expression, in contexts such as cell differentiation, transitions, and perturbations.


## Dataset description
 Cell-type clusters of 100,000 human PBMCs and the NeurIPS dataset, which contain healthy donors as well as COVID-19 patients{cite}`Schulte-chrepping2020`.

### Installation of SCENIC

The package SCENIC requires a variety of tool that are used for the annotation of TF-regulons and the calculation of strong edges between TFs and target genes (see regulons_only_rna.yml for installation requirements)

## Environment setup

In [4]:
import warnings

# here verify all the relevant import that will be required.
warnings.filterwarnings("ignore")
import pyscenic
import loompy as lp
import scanpy as sc
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
import os

## Preparation of the NeurIPs dataset
The current dataset we will be looking here is already pre-processed (previous chapters), and it contains 69,249 cells, annotated into 22 cell types. Due to batch integration considerations, we will demonstrate the usage of GRN methods for only one, labeled as s1d1 (n=6,224 cells).

Load the full dataset

In [5]:
adata = sc.read_h5ad("../../data/openproblems_bmmc_multiome_genes_filtered.h5ad")
adata.shape

KeyboardInterrupt: 

Subset RNA only features using the label GEX

In [ ]:
rna = adata[:, adata.var.feature_types == "GEX"]
del adata
sc.pp.log1p(rna)
rna.obs.batch.value_counts()

In [ ]:
rna.shape

Here we select highly-variable genes, to minimize the computing requirements for steps down-stream. An execution without defining HVGs is also possible, but will need additional computing memory and time.

In [ ]:
sc.pp.highly_variable_genes(rna, batch_key="batch", flavor="seurat")

This is the embedding of all gene expression data cells of all donors. One can see that donor is dominating over cell types, due to batch-correction methods not executed.

In [ ]:
sc.set_figure_params(facecolor='white')

In [ ]:
sc.pl.embedding(rna, "GEX_X_umap", color=["cell_type", "batch"])

Here we observe the cells of only donor s1d1. Cells cluster well based on their provided annotations

In [ ]:
adata_batch = rna[rna.obs.batch == "s1d1", :]
print("custom donor (s1d1)")
sc.pl.embedding(adata_batch, "GEX_X_umap", color=["cell_type", "batch"])

## Preparation of SCENIC

Define the input for SCENIC using loompy

In [ ]:
## this file has to be downloaded if not found
!wget -nc https://raw.githubusercontent.com/aertslab/SCENICprotocol/master/example/allTFs_hg38.txt

In [ ]:
tfs_path = "allTFs_hg38.txt"

In [ ]:
loom_path = "data/neurips_processed_input.loom"
loom_path_output = "data/neurips_processed_output.loom"
tfs = [tf.strip() for tf in open(tfs_path)]

# check that the needed output file (before execution it should be False/False
print(os.path.exists(loom_path), os.path.exists(loom_path_output))

As a verification, it is recommended to check that genes annotated as TFs are part of the provided input data. If not a high coverage of gene symbols is observed e.g. 50% or more, there could be an issues with the feature names declared in the `.var`, such as Ensembl IDs instead of gene_symbols, or lower/uppercase difference if the genome assembly is mouse or another species that uses a different capitalization style than human.

In [ ]:
# as a general QC. We inspect that our object has transcription factors listed in our main annotations.
print(
    np.sum(adata_batch.var.index.isin(tfs)),
    "out of",
    len(tfs),
    "TFs are found in the object",
)

Usage of HVG or all features can be defined here, by changing the flag `use_hvg` to `False`.

In [ ]:
use_hvg = True
if use_hvg:
    mask = (adata_batch.var["highly_variable"] == True) | adata_batch.var.index.isin(
        tfs
    )
    adata_batch = adata_batch[:, mask]

Create a loom file for NeurIPS donor. If you get an error in this step, verify thath the labels for Gene/CellID/etc. are properly defined.

In [ ]:
row_attributes = {
    "Gene": np.array(adata_batch.var.index),
}
col_attributes = {
    "CellID": np.array(adata_batch.obs.index),
    "nGene": np.array(np.sum(adata_batch.X.transpose() > 0, axis=0)).flatten(),
    "nUMI": np.array(np.sum(adata_batch.X.transpose(), axis=0)).flatten(),
}

lp.create(loom_path, adata_batch.X.transpose(), row_attributes, col_attributes)

Once the loom file has been generated, we execute `pyscenic` to generate associations between TFs and genes. TF-gene associations are inferred by GRNBoost, and summarized by a directional weight between TFs and target genes. The output of this analysis is a table summarizing all reported associations with their importance weight.

Some steps below require indicating a number of cores (`num_workers`). Increase according to computing resources available

In [ ]:
num_workers = 3

In [ ]:
outpath_adj = "adj.csv"
if not os.path.exists(outpath_adj):
    !pyscenic grn {loom_path} {tfs_path} -o $outpath_adj --num_workers {num_workers}

Show the top of of TF-target associations

In [ ]:
results_adjacencies = pd.read_csv("adj.csv", index_col=False, sep=",")
print("# associations", results_adjacencies.shape)
results_adjacencies.head()

Visualize the distribution of weights for general inspection of the quantiles and thresholds obtained from pyscenic

In [ ]:
plt.hist(np.log10(results_adjacencies["importance"]), bins=50)
plt.xlim([-10, 10])

Targets genes have DNA motifs at promoters (sequence specific DNA motifs), those can be used to link TFs to target genes. Here we use an annotation of TFs as Transcription Start Sites (TSSs) to refine this annotation.

Download TSS annotations

In [ ]:
!wget -nc https://resources.aertslab.org/cistarget/databases/homo_sapiens/hg38/refseq_r80/mc9nr/gene_based/hg38__refseq-r80__10kb_up_and_down_tss.mc9nr.feather

In [ ]:
import glob
import os

# ranking databases
db_glob = "*feather"
db_names = " ".join(glob.glob(db_glob))

Download a catalog of motif-to-TF associations

In [ ]:
!wget -nc https://resources.aertslab.org/cistarget/motif2tf/motifs-v9-nr.hgnc-m0.001-o0.0.tbl

In [ ]:
# motif databases
motif_path = "motifs-v9-nr.hgnc-m0.001-o0.0.tbl"
assert os.path.exists(motif_path)

Using the catalog of motif and their gene associations at promotoers, retrieved a subset adjacencies by pruning of the previous adjacencies.
This step can take ~8 minutes in a laptop.

In [ ]:
if not os.path.exists('reg.csv'):
    !pyscenic ctx adj.csv \
        {db_names} \
        --annotations_fname {motif_path} \
        --expression_mtx_fname {loom_path} \
        --output reg.csv \
        --mask_dropouts \
        --num_workers {num_workers} > pyscenic_ctx_stdout.txt

To explore the candidates reported, it is recommended as a rule of thumb to explore the output by the ranking of relative contribution, or by top-quantile thresholds defined visually, to obtain a high signal-to-noise ratio.

Define custom quantiles for further exploration

In [ ]:
import numpy as np

nGenesDetectedPerCell = np.sum(adata_batch.X > 0, axis=1)
percentiles = pd.Series(nGenesDetectedPerCell.flatten().A.flatten()).quantile(
    [0.01, 0.05, 0.10, 0.50, 1]
)
print(percentiles)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 5), dpi=100)
sns.distplot(nGenesDetectedPerCell, norm_hist=False, kde=False, bins="fd")
for i, x in enumerate(percentiles):
    fig.gca().axvline(x=x, ymin=0, ymax=1, color="red")
    ax.text(
        x=x,
        y=ax.get_ylim()[1],
        s=f"{int(x)} ({percentiles.index.values[i]*100}%)",
        color="red",
        rotation=30,
        size="x-small",
        rotation_mode="anchor",
    )
ax.set_xlabel("# of genes")
ax.set_ylabel("# of cells")
fig.tight_layout()

This step will use TFs to calculate Area Under the Curve scores, that summarize how well the gene expression observed in each cell can be associated by the regulation of target genes regulatred by the mentioned TFs.

In [ ]:
import json
import zlib
import base64
# collect SCENIC AUCell output
lf = lp.connect(loom_path_output, mode="r+", validate=False)
auc_mtx = pd.DataFrame(lf.ca.RegulonsAUC, index=lf.ca.CellID)
lf.close()

Using the above-generated matrix of cell x TFs and those scores, we can calculate a new embedding using only those.

In [ ]:
from MulticoreTSNE import MulticoreTSNE as TSNE

In [ ]:
if not os.path.exists(loom_path_output):
    !pyscenic aucell $loom_path \
        reg.csv \
        --output {loom_path_output} \
        --num_workers {num_workers} > pyscenic_aucell_stdout.txt

In [ ]:
import umap

# UMAP
runUmap = umap.UMAP(n_neighbors=10, min_dist=0.4, metric="correlation").fit_transform
dr_umap = runUmap(auc_mtx)
pd.DataFrame(dr_umap, columns=["X", "Y"], index=auc_mtx.index).to_csv(
    "scenic_umap.txt", sep="\t"
)

In [ ]:
# tSNE
tsne = TSNE(n_jobs=20)
dr_tsne = tsne.fit_transform(auc_mtx)

In [ ]:
df_tsne = pd.DataFrame(dr_tsne, columns=["X", "Y"], index=auc_mtx.index)
df_tsne.to_csv("scenic_tsne.txt", sep="\t")

In [ ]:
df_tsne.shape, adata_batch.shape

Visualize the data base on the TF-regulons and auc_mtx generated.

In [ ]:
sc.pp.neighbors(adata_batch)
sc.tl.umap(adata_batch)

In [ ]:
sc.pl.umap(adata_batch, color="cell_type")

In [ ]:
adata_batch.obsm["X_umap"] = df_tsne.to_numpy()
sc.pl.umap(adata_batch, color="cell_type")

### **Interpretation of results**

In [ ]:
import seaborn as sns

In [ ]:
auc_mtx["cell_type"] = adata_batch.obs["cell_type"]
mean_auc_by_cell_type = auc_mtx.groupby("cell_type").mean()

show the top N TF regulons by a color

In [ ]:
top_n = 50
top_tfs = mean_auc_by_cell_type.max(axis=0).sort_values(ascending=False).head(top_n)
mean_auc_by_cell_type_top_n = mean_auc_by_cell_type[
    [c for c in mean_auc_by_cell_type.columns if c in top_tfs]
]

In [ ]:
sns.clustermap(
    mean_auc_by_cell_type_top_n,
    figsize=[15, 6.5],
    cmap="Blues",
    xticklabels=True,
    yticklabels=True,
)

This heatmap visualization is suggesting that some TFs are strongly associated to particular cell types. We can verify the expression of their expression levels as an additional validation.

top

In [ ]:
tf_names = top_tfs.index.str.replace("\(\+\)", "")
adata_batch_top_tfs = adata_batch[:, adata_batch.var_names.isin(tf_names)]

In [ ]:
sc.pl.matrixplot(
    adata_batch,
    tf_names,
    groupby="cell_type",
    cmap="Reds",
    dendrogram=True,
    figsize=[15, 5.5],
    standard_scale="group",
)

Through visual inspection and comparison of the TF-regulon AUC scores and TF gene expression by cell type, we can verify that in several case the cell-type specific expression of a TF is linked to a particular cell type, where the respective TF-regulon is also active e.g. RUNX2 in pDCs, TCF7L2 in CD16+ Mono, LEF1 in CD4+ T activated. Additional inspection can serve to validate previous insights and/or additional associations.

## Preparation of data for RNA+ATAC notebook

Calculations in the next notebook require prior co-variation weights calculated for ATAC/RNA features, that will be done here for convenience.

Preparation of **RNA** for RNA+ATAC notebook (s1d1)

In [ ]:

cd workspace/theislab/extended-single-cell-best-practices/jupyter-book/mechanisms

In [ ]:
import scanpy as sc
import os
from matplotlib import rcParams
import pandas as pd

path_s1d1 = "../../data/openproblems_bmmc_multiome_genes_filtered_s1d1.h5ad"
# print(os.path.exists(path_s1d1), path_s1d1)
if not os.path.exists(path_s1d1):
    adata = sc.read_h5ad("../../data/openproblems_bmmc_multiome_genes_filtered.h5ad")
    adata_s1d1 = adata[adata.obs.batch == "s1d1", :]
    adata_s1d1.write(path_s1d1, compression="lzf")
    del adata

adata_s1d1 = sc.read_h5ad(path_s1d1)

Setting for plotting of UMAPs with a white background and low dpi

In [ ]:
from matplotlib import rcParams

rcParams["figure.facecolor"] = "white"
rcParams["figure.figsize"] = 5, 5
rcParams["figure.dpi"] = 65

In [ ]:
rna = adata_s1d1[:, adata_s1d1.var.feature_types == "GEX"]
sc.pp.log1p(rna)
rna.obs.batch.value_counts()

print("custom batch (input")
sc.pl.embedding(rna, "GEX_X_umap", color=["cell_type", "batch"])

In [ ]:
# define hvgs in RNA
sc.pp.highly_variable_genes(rna)

In [ ]:
rna.var['transcription_factor'] = rna.var.index.isin(tfs)
rna.var.transcription_factor.value_counts()

In [ ]:
rna_hvg_sorted = rna[:,rna.var.sort_values(['transcription_factor', 'dispersions_norm'], ascending=[False, False]).index].copy()

In [ ]:
# rna_dense = pd.DataFrame(data=rna.X.todense())
# rna.var.to_csv('../../data/openproblems_bmmc_multiome_genes_filtered_rna_s1d1_var.csv.gz', index=True, compression='gzip')
rna_hvg_sorted.to_df().to_csv(
    "../../data/openproblems_bmmc_multiome_genes_filtered_rna_s1d1.csv.gz",
    compression="gzip",
)

Preparation of **ATAC** for RNA+ATAC notebook (donor **s1d1**)

In [ ]:
atac = adata_s1d1[:, adata_s1d1.var.feature_types == "ATAC"]

In [ ]:
n_features_atac = 20_000
if n_features_atac is not None:
    sc.pp.highly_variable_genes(atac, n_top_genes=n_features_atac)
    atac = atac[:, atac.var.highly_variable]
atac

In [ ]:
print("custom batch (input)")

from matplotlib import rcParams

rcParams["figure.figsize"] = 5, 5
rcParams["figure.facecolor"] = 'white'

sc.pl.embedding(atac, "ATAC_umap", color=["cell_type", "batch"])

In [ ]:
atac_obs = atac.obs
atac_obs["UMAP1"] = atac.obsm["ATAC_umap"][:, 0]
atac_obs["UMAP2"] = atac.obsm["ATAC_umap"][:, 1]

In [ ]:
atac_obs.to_csv(
    "../../data/openproblems_bmmc_multiome_genes_filtered_atac_s1d1_obs.csv.gz"
)

In [ ]:
atac_df = atac.to_df().T

atac_df["chr"] = atac_df.index.str.split("-").str[0]
atac_df["start"] = atac_df.index.str.split("-").str[1]
atac_df["end"] = atac_df.index.str.split("-").str[2]

# locate at the beginning of the dataframe
for i, c in enumerate(["chr", "start", "end"]):
    print(i, c)
    column_to_move = atac_df.pop(c)
    atac_df.insert(i, c, column_to_move)
atac_df = atac_df.T
print(atac_df.shape)

In [ ]:
atac_bkp_path = (
    "../../data/openproblems_bmmc_multiome_genes_filtered_atac_s1d1_counts_features_times_cells%s.csv.gz"
    % ("" if n_features_atac is None else ("_npeaks" + str(n_features_atac)))
)
if not os.path.exists(atac_bkp_path):
    atac_df.T.to_csv(atac_bkp_path, compression="gzip")
print(atac_df.T.shape, os.path.exists(atac_bkp_path), atac_bkp_path)

## Takeaways

In this notebook, we have:

1. Setup a basic pipeline to prepare, execute, and verify SCENIC results using the NeurIPs 2021 dataset.
2. Detected TF-regulons that allow to explain the observed transcriptomes by the regulatory potential of a handful of TFs.


## Quiz

### Theory
1. How many genes in the human/mouse genome are considered TFs?
2. What is a TF-regulon?
3. How many DNA-binding motifs are there to each TFs?
    3.1. Currently, are there more known DNA-binding motifs or TFs that bind those?
    3.2. How can one reconcile the redundancy of these during the analysis?
4. Additional reading: Describe the futility theorem.

### SCENIC
1. Describe the major steps in the SCENIC pipeline (three or more)?
2. What could happen if the number of TFs reported in your own data has a low overlap to the one externally retrieved?
3. What is the AUC score from pyscenic and how it can be helpful for interpretation of results per cell cluster?

## References

```{bibliography}\n",
:filter: docname in docnames
:labelprefix: anno
```

## Contributors
We gratefully acknowledge the contributions of:
### Authors
* Ignacio Ibarra
### Reviewers
* Lukas Heumos
* Anna Schaar